In [1]:
%load_ext memory_profiler

## Load Dependancies 

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR

from sklearn import datasets
import scipy.stats as stats
import lightgbm as lgb
import xgboost as xgb

from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import numpy as np

import dask
from dask.distributed import worker_client
from dask.distributed import Client,LocalCluster

import joblib
import numpy as np
from dask.distributed import Client

import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from xgboost import XGBRegressor

In [3]:
client = Client(scheduler_file='scheduler.json')
client

Client Scheduler: tcp://10.6.21.67:8787 Dashboard: http://10.6.21.67:8791/status,Cluster Workers: 96 Cores: 96 Memory: 404.96 GB


In [4]:
dask.config.set(scheduler='threads')


## Load Data

In [5]:
root = Path('/scratch/ge3/sg4953') 

In [6]:
# read the csv file created
train = np.genfromtxt(str(root)+'/input_dataset.csv', delimiter=',')
train = train.astype(np.float32)
train = train[~np.isnan(train).any(axis=1)]
train = train[~(train==-9999.0).any(axis=1)]

X, y = train[:, 1:], train[:, :1]
y = y.ravel()


In [7]:
# Set training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
# check for nodata values
# for i in range (train.shape[0]):
#     for j in range(train.shape[1]):
#         if train[i][j] == -9999.0:
#             print(i,j,train[i][j])
        

In [9]:
%%time
%%memit
#XGBoost


{'target': nan,
 


params={
    "max_depth":5, 
    "min_child_weight":2, 
    "eta": 1, 
    "colsample_bytree":0.8,  
    "eval_metric": "auc",
    'objective':'reg:squarederror'
}

clf = xgb.XGBModel(**params)


peak memory: 252.43 MiB, increment: 0.02 MiB
CPU times: user 88.6 ms, sys: 43.1 ms, total: 132 ms
Wall time: 281 ms


In [14]:
%%time
input_file = Path("../data/quantile_df.csv")

df = pd.read_csv(input_file)
df = df.drop(["x","y"],axis=1)

y = df['target'] 
X = df.drop("target",axis=1)

model = XGBRegressor(objective='reg:squarederror',tree_method='gpu_hist', gpu_id=0, n_jobs=-1,gamma=1,learning_rate=5,max_depth=20,n_estimators=100)
cv_scores = cross_val_score(model, X, y, scoring='r2', n_jobs=-1)
mean_score = np.mean(cv_scores)
print("mean_score",mean_score)

mean_score nan
CPU times: user 3.2 s, sys: 151 ms, total: 3.35 s
Wall time: 12min 17s


In [ ]:
mean_score 0.465343944571125